In [1]:
import pandas as pd
import random
import numpy as np
import datetime

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
INPUT_FILE = '../toxicity_ratings - twitter.csv'

In [4]:
df = pd.read_csv(INPUT_FILE)
len(df)

/Users/shagunjhaver/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,7,8,9,10,14,15,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


72947

In [5]:
df = df.dropna()
len(df)

70208

In [6]:
toxic_fields = [
    'toxic_score',
    'toxic_score.1',   
    'toxic_score.2',
    'toxic_score.3',    
    'toxic_score.4',    
]

In [8]:
def get_avg_toxicity(row):
    myList = [row[field] for field in toxic_fields]
    result = sum(myList)/20.0
    for field in toxic_fields:
        maxValue = max(myList)
        minValue = min(myList)
        if (maxValue - minValue > 1):
            return None
    return result

In [9]:
df['avg_toxic_score'] = df.apply (lambda row: get_avg_toxicity(row), axis=1)

In [10]:
df = df.dropna()
len(df)

27994

In [11]:
def get_bucket(row, field):
    if (row[field] < 0.6):
        return 0.0
    elif (row[field] < 0.7):
        return 0.6
    elif (row[field] < 0.8):
        return 0.7
    elif (row[field] < 0.9):
        return 0.8
    else:
        return 0.9

In [12]:
df['avg_toxic_window'] = df.apply(lambda row: get_bucket(row, 'avg_toxic_score'), axis=1)

In [13]:
myrange = [0.6, 0.7, 0.8, 0.9]
sampled_dfs = []
for ow in myrange:
    g = df[df['avg_toxic_window'] == ow]
    print (ow, len(g))
    sampled_dfs.append(g.sample(10))

0.6 111
0.7 30
0.8 61
0.9 82


In [14]:
final_df = pd.concat(sampled_dfs)

In [15]:
def get_random_date():
    random_number_of_days = random.randrange(40)
    random_date = datetime.date(2021, 10, 1) + datetime.timedelta(days=random_number_of_days)
    return random_date

In [16]:
def assignPeripherals(final_df):
    final_df = final_df[['comment', 'avg_toxic_score', 'comment_ID']]
    
    # Set username
    usernameDf = pd.read_csv('usernames.csv')
    usernames = set(list(usernameDf['username']))
    final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
    
    # Set full name
    nameDf = pd.read_csv('random_names.csv')
    names = set(list(nameDf['name']))
    final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)    
    
    # Set integer fields
    final_df['num_likes'] = np.random.randint(1, 100, final_df.shape[0])    
    final_df['num_comments'] = np.random.randint(1, 100, final_df.shape[0])    
    final_df['num_retweets'] = np.random.randint(1, 100, final_df.shape[0])        
    
    # Set publication date
    final_df['pub_date'] = df.apply(lambda row: get_random_date(), axis=1)
    
    return final_df

In [17]:
final_df = assignPeripherals(final_df)
final_df.head()

<ipython-input-16-61dd48437cb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
<ipython-input-16-61dd48437cb0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)
<ipython-input-16-61dd48437cb0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,comment,avg_toxic_score,comment_ID,author,author_name,num_likes,num_comments,num_retweets,pub_date
11628,Fast &amp; Furious 9: its not my fault youre ugly man sorry to break it to you,0.6,111_18,PM_ME_YOUR_MAYORS,Preston Owens,13,2,71,2021-10-21
27474,Lovely peaches be getting that dick 🙈,0.6,2435_16,favorableheads3963,Elian Owens,45,31,48,2021-10-26
10234,"Lots of people think about your Dad every day. They think he is everything the British Ambassador said he is, beyond being a liar, buffoon, shallow as a dog-pee puddle, treasonous-Russian asset &amp; hopeless failure at business. Be grateful for the short time he has before prison.",0.6,1280_8,MustySeating,Lucia Reed,10,82,52,2021-10-10
2806,This isn't america can everyone fuck off eith this racist and blame the police nonsense. Stop acting like cu ts and tou wont be arrested. Its your actions which are wrong and you all need to learn some humility.,0.6,1093_23,VirtualStatement,Alexia Wilson,54,69,84,2021-10-26
56065,The fact that #NotMyAriel is trending proves how evil white ppl are as a whole and Narcissistic.,0.6,5113_1,thefrizzyforgiveness,Eddy Rogers,17,99,78,2021-11-05


In [18]:
final_df = final_df.sample(frac=1).reset_index(drop=True)
final_df.to_csv('main_comments.csv', index=False)

In [19]:
nontoxic_df = df.query('avg_toxic_window == 0.0')

In [20]:
nontoxic_sample = nontoxic_df.sample(30)

In [21]:
nontoxic_sample = assignPeripherals(nontoxic_sample)
nontoxic_sample.head()

<ipython-input-16-61dd48437cb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
<ipython-input-16-61dd48437cb0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)
<ipython-input-16-61dd48437cb0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,comment,avg_toxic_score,comment_ID,author,author_name,num_likes,num_comments,num_retweets,pub_date
46029,I would be lost &amp; broke In a ditch without my sister,0.00,2291_1,intrigued_tights,Ashton Campbell,3,76,81,2021-10-27
34500,Your relationships pose challenges to your self-expression now... More for Cancer,0.05,3151_7,LastVictory,Lyndon Barrett,18,30,85,2021-10-03
7464,"Thank you! Like yeah Adam is a killer and shit but guess what? So is Cinder, Neo, Mercury, Emerald, Salem and every single other villain in this show and people like who they like but that doesn't mean they condone their actions or support the evil choices they make.",0.00,814_9,rebelsolitude666,Jasmine Montgomery,11,9,24,2021-10-10
57278,"I refuse to ever fall in love again. . The thought of letting someone back in scares me or even liking someone scares the shit out of me, love isn’t made for me and never will be.",0.00,4370_20,moist_hello,Lyndon Warren,31,70,2,2021-11-03
10372,A mere tweet cannot prevent me from giving birth. And I don't go about bashing people because I don't like them. I wasn't trained like dat.... You can support your fav without insulting others. ✌,0.05,306_6,exciting_self-control,Byron Alexander,89,37,90,2021-10-12


In [22]:
nontoxic_sample.to_csv('nontoxic_comments.csv', index=False)